In [8]:
import sys
sys.path.append("../")

In [9]:
import pandas as pd
import datetime as dt
from technical.indicators import rsi
from technical.patterns import apply_patterns
from simulation.guru_tester import GuruTester

In [10]:
BUY = 1
SELL = -1
NONE = 0
RSI_LIMIT = 50.0

def apply_signal(row):
    if row.ENGULFING == True:
        if row.direction == BUY and row.mid_l > row.EMA_200:
            if row.RSI_14 > RSI_LIMIT:
                return BUY
        if row.direction == SELL and row.mid_h < row.EMA_200:
            if row.RSI_14 < RSI_LIMIT:
                return SELL
    return NONE        

In [11]:
def run_pair(pair):
    df_an = pd.read_pickle(f"../data/candles/{pair}_H1.pkl")
    df_m5 = pd.read_pickle(f"../data/candles/{pair}_M5.pkl")
    df_an.reset_index(drop=True, inplace=True)
    df_m5.reset_index(drop=True, inplace=True)
    df_an = rsi(df_an)
    df_an = apply_patterns(df_an)
    df_an['EMA_200'] = df_an.mid_c.ewm(span=200, min_periods=200).mean()
    our_cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c',
            'bid_o', 'bid_h', 'bid_l', 'bid_c', 
            'ask_o', 'ask_h', 'ask_l', 'ask_c',
            'ENGULFING', 'direction', 'EMA_200', 'RSI_14' ]
    df_slim = df_an[our_cols].copy()
    df_slim.dropna(inplace=True)
    
    df_slim.reset_index(drop=True, inplace=True)
    gt = GuruTester(
        df_slim,
        apply_signal,
        df_m5,
        use_spread=True
    )
    
    gt.run_test()
    return gt.df_results
    

In [12]:
res = []
for p in ["EUR_USD", "GBP_JPY"]:
    res.append(dict(pair=p,res=run_pair(p)))

prepare_data...
run_test...
Result: -1016.0
prepare_data...
run_test...
Result: -1057.5


In [13]:
for r in res:
    print(r['pair'], r['res'].result.sum())

EUR_USD -1016.0
GBP_JPY -1057.5
